In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samonkaggle","key":"36f76ca324bc87aca9d014433b896a18"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json  ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                 title                                  size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  ------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                  Data Science Job Salaries               7KB  2022-06-15 08:59:12          17624        549  1.0              
aravindas01/monkeypox-cases-countrywise-data        MonkeyPox Cases_Countrywise Data        6KB  2022-08-10 17:12:36            525         27  0.9117647        
faryarmemon/usa-housing-market-factors              U.S. Housing Market Factors            32KB  2022-08-03 02:19:31            425         30  1.0              
zzettrkalpakbal/full-filled-brain-stroke-dataset    Brain stroke prediction dataset        52KB  2022-07-16 09:57:08           1973         62  0.9705882        
himanshunakrani/student-stud

In [ ]:
! kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 71% 29.0M/41.0M [00:00<00:00, 154MB/s] 
100% 41.0M/41.0M [00:00<00:00, 154MB/s]


In [ ]:
!unzip fake-and-real-news-dataset.zip

Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [ ]:
real_data=pd.read_csv("True.csv")
fake_data=pd.read_csv("Fake.csv")

In [ ]:
#Removing Date Attribute
real_data = real_data.iloc[:, 0:-1] 
fake_data = fake_data.iloc[:, 0:-1] 

In [ ]:
print(real_data.shape)
print(fake_data.shape)

(21417, 3)
(23481, 3)


In [ ]:
fake_rate=(fake_data.shape[0])/((fake_data.shape[0])+real_data.shape[0])*100
print("Percentage of Fake data is: ", fake_rate)

Percentage of Fake data is:  52.29854336496058


In [ ]:
fake_data['label']=0
real_data['label']=1

In [ ]:
data = fake_data
data=pd.concat([fake_data,real_data], axis=0, ignore_index=True)
data

,title,text,subject,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,0
...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,1


In [ ]:
data_title_label = data.iloc[:, [0,3]]

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 44898):
    title = re.sub('[^a-zA-Z]', ' ', data_title_label['title'][i]) # review to contain only letters a to z and if the character is removed replace it by so+pace
    title = title.lower() # review to contain only lower case letter
    title = title.split() # Splitting the string into different words
    ps = PorterStemmer() # Creating object of the class
    title = [ps.stem(word) for word in title if not word in set(stopwords.words('english'))]  # Set here is used only for faster execution of code in case there are too many words...
    title = ' '.join(title) # Converting the list back to string
    corpus.append(title)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=15000)
X = cv.fit_transform(corpus).toarray()
y = data_title_label.iloc[:, 1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.25,
                                                   random_state=0)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input, Embedding, LSTM, Concatenate

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) 
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) 

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
ann.fit(X_train, y_train, batch_size=32, epochs=100, verbose=2,callbacks=[monitor])




Epoch 1/100
1053/1053 - 12s - loss: 0.0498 - accuracy: 0.9824 - 12s/epoch - 12ms/step
Epoch 2/100
1053/1053 - 9s - loss: 0.0401 - accuracy: 0.9862 - 9s/epoch - 8ms/step
Epoch 3/100
1053/1053 - 8s - loss: 0.0330 - accuracy: 0.9891 - 8s/epoch - 8ms/step
Epoch 4/100
1053/1053 - 8s - loss: 0.0289 - accuracy: 0.9898 - 8s/epoch - 8ms/step
Epoch 5/100
1053/1053 - 8s - loss: 0.0245 - accuracy: 0.9910 - 8s/epoch - 8ms/step
Epoch 6/100
1053/1053 - 8s - loss: 0.0202 - accuracy: 0.9926 - 8s/epoch - 8ms/step
Epoch 7/100
1053/1053 - 8s - loss: 0.0167 - accuracy: 0.9942 - 8s/epoch - 8ms/step
Epoch 8/100
1053/1053 - 8s - loss: 0.0142 - accuracy: 0.9948 - 8s/epoch - 8ms/step
Epoch 9/100
1053/1053 - 8s - loss: 0.0110 - accuracy: 0.9960 - 8s/epoch - 8ms/step
Epoch 10/100
1053/1053 - 8s - loss: 0.0094 - accuracy: 0.9965 - 8s/epoch - 8ms/step
Epoch 11/100
1053/1053 - 8s - loss: 0.0078 - accuracy: 0.9972 - 8s/epoch - 8ms/step
Epoch 12/100
1053/1053 - 8s - loss: 0.0060 - accuracy: 0.9980 - 8s/epoch - 8ms/ste

In [ ]:
y_pred = ann.predict(X_test)

In [ ]:
y_pred = (y_pred>0.5)

In [ ]:
y_pred

array([[ True],
       [ True],
       [ True],
       ...,
       [False],
       [ True],
       [False]])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[5489,  387],
       [ 375, 4974]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      5876
           1       0.93      0.93      0.93      5349

    accuracy                           0.93     11225
   macro avg       0.93      0.93      0.93     11225
weighted avg       0.93      0.93      0.93     11225



In [ ]:
X.shape

(44898, 13206)

In [ ]:
import pickle

In [ ]:
pickle.dump(ann, open('model.pkl', 'wb'))

In [ ]:
pickled_model = pickle.load(open('model.pkl', 'rb'))
pickled_model.predict(X_test)

array([[1.0000000e+00],
       [9.9998647e-01],
       [1.0000000e+00],
       ...,
       [7.6403409e-02],
       [1.0000000e+00],
       [5.4982303e-29]], dtype=float32)

In [ ]:
import urllib.request
from xml.dom.minidom import parseString

def get_google_new_results( term, count ):
    results = []
    terms=term.split()
    ss=""
    ss=ss.join(terms)
    obj = parseString( urllib.request.urlopen('http://news.google.com/news?q=%s&output=rss' % ss).read() )
    items = obj.getElementsByTagName('item') # Get each item
    for item in items[:count]:
        t,l = '', ''
        for node in item.childNodes:
            if node.nodeName == 'title':
                t = node.childNodes[0].data
            elif node.nodeName == 'link':
                l = node.childNodes[0].data
        results.append( (t,l) )
    return results


print("Enter term to scrape from")
x = str("Donald Trump")

items = get_google_new_results( x, 50 )

# df.append()
# new_df = pd.DataFrame(columns=["TITLE","LINK"])
result=[]
for title,link in items:
  result.append(title)

# for X_new in result:
#   corpus = []
#   title = re.sub('[^a-zA-Z]', ' ', X_new) # review to contain only letters a to z and if the character is removed replace it by so+pace
#   title = title.lower() # review to contain only lower case letter
#   title = title.split() # Splitting the string into different words
#   ps = PorterStemmer() # Creating object of the class
#   title = [ps.stem(word) for word in title if not word in set(stopwords.words('english'))]  # Set here is used only for faster execution of code in case there are too many words...
#   title = ' '.join(title) # Converting the list back to string
#   corpus.append(title)

Enter term to scrape from


In [ ]:
X_new=[]
for i in result:
  corpus = []
  title = re.sub('[^a-zA-Z]', ' ', i) # review to contain only letters a to z and if the character is removed replace it by so+pace
  title = title.lower() # review to contain only lower case letter
  title = title.split() # Splitting the string into different words
  ps = PorterStemmer() # Creating object of the class
  title = [ps.stem(word) for word in title if not word in set(stopwords.words('english'))]  # Set here is used only for faster execution of code in case there are too many words...
  title = ' '.join(title) # Converting the list back to string
  corpus.append(title)
  X_new.append(corpus)
print(len(X_new))

50


In [ ]:
result

['Former President Donald Trump invokes Fifth Amendment rights and declines to answer questions from NY attorney general - CNN',
 'Analysis | Donald Trump has been preparing for this moment for a long time - The Washington Post',
 'The vise is tightening around Donald Trump as 2024 decision looms - CNN',
 'How to Prosecute Donald Trump - National Review',
 'How Donald Trump changed Facebook - The Verge',
 "Donald Trump on 2024: 'I've Already Made That Decision' - New York Magazine",
 'Opinion | Donald Trump’s Politics of Persecution - The New York Times',
 'Trump Will Release the Warrant - The Atlantic',
 'The Cult of Donald Trump - The Intercept',
 'Liz Cheney’s Revenge on Donald Trump—and Her Own Party - The New Yorker',
 "'Posturing and politicking' will lead to US resembling third-world country: Attorney for Donald Trump - Fox News",
 'Donald Trump hints at 2024 White House comeback bid: ‘The time is coming’ - Fox News',
 'Donald Trump Teases 2024 Run in Return to Washington - Bloo

In [ ]:
#Transforming the the headline into bag of words with pretrained weights
final_pred=[]
confidence=[]
for i in X_new:
  X_sample = cv.transform(i).toarray()
  prob=pickled_model.predict(X_sample.reshape(1, -1))
  if(prob>0.5):
    final_pred.append("True News")
    confidence.append(int(prob*100))
    # print(float(prob*100))
    # print("True News")
  else:
    final_pred.append("Fake News")
    confidence.append(int(prob*100))
    # print(float(prob*100))
    # print("Fake News")

In [ ]:
conf=pd.DataFrame(confidence,columns=["Confidence"])

In [ ]:
dff=pd.DataFrame(final_pred, columns=["Final Prediction"])

In [ ]:
final_titles=pd.DataFrame(result,columns=["Tilte"])

In [ ]:
final_display=pd.concat([final_titles, dff,conf], axis=1, join='inner')

In [ ]:
final_display

,Tilte,Final Prediction,Confidence
0,Former President Donald Trump invokes Fifth Am...,True News,99
1,Analysis | Donald Trump has been preparing for...,True News,99
2,The vise is tightening around Donald Trump as ...,True News,99
3,How to Prosecute Donald Trump - National Review,Fake News,2
4,How Donald Trump changed Facebook - The Verge,True News,84
5,Donald Trump on 2024: 'I've Already Made That ...,Fake News,0
6,Opinion | Donald Trump’s Politics of Persecuti...,Fake News,0
7,Trump Will Release the Warrant - The Atlantic,True News,99
8,The Cult of Donald Trump - The Intercept,Fake News,0
9,Liz Cheney’s Revenge on Donald Trump—and Her O...,True News,99


In [ ]:
fake_news_percent=(final_display["Final Prediction"].value_counts()["Fake News"])*(100/50)

In [ ]:
fake_news_percent

68.0